In [2]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.applications import Xception, VGG16, MobileNet
from tensorflow.keras.layers import Flatten, Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [3]:
img_width, img_height = 224, 224

base_model = Xception(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))
# pre-trained model : Xceptions
# include_top=False : 사전학습모델의 특징 추출기만가져옴 / True : 사전학습모델의 특징추출기와 분류기 모두 가져옴
model = Sequential()

model.add(base_model)

model.add(GlobalAveragePooling2D())

# user-defined classifier
model.add(Dense(16, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(2, activation='softmax')) # 정답은 'cat','dog' 2개이므로 출력층 노드는 2개

model.summary()

83683744/83683744 [==============================] - 6s 0us/step
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 xception (Functional)       (None, 7, 7, 2048)        20861480  
                                                                 
 global_average_pooling2d (G  (None, 2048)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 16)                32784     
                                                                 
 dropout (Dropout)           (None, 16)                0         
                                                                 
 dense_1 (Dense)             (None, 2)                 34        
                                                                 
Total params: 20,894,298
Trainable params: 20,839,770
Non

## 모델 전처리

In [11]:
# ImageDataGenerator 정의
train_dir = '/Users/민동률/DL/data/datasets_small/train'
test_dir = '/Users/민동률/DL/data/datasets_small/test'

# rescale=1./255 : 이미지를 읽어올때 자동으로 정규화됨

train_data_gen = ImageDataGenerator(rescale=1./255, rotation_range=10, width_shift_range=0.1,
                                    height_shift_range=0.1, shear_range=0.1, zoom_range=0.1)

test_data_gen = ImageDataGenerator(rescale=1./255)

In [12]:
# categorical이므로 정답은 one-hot encoding으로 정의됨

train_data = train_data_gen.flow_from_directory(train_dir, batch_size=32, color_mode = 'rgb', shuffle=True, class_mode = 'categorical',
                                                target_size=(img_width, img_height))

test_data = test_data_gen.flow_from_directory(test_dir, batch_size=32, color_mode = 'rgb', shuffle=True, class_mode = 'categorical',
                                              target_size=(img_width, img_height))


Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [13]:
# 정답 확인
print(train_data.class_indices.items()) 
# class_indices속성을 이용하면, 문자열로 표시되는 데이터의 정답이 어떤 숫자로 매칭되어 있는지 확인 가능
print(test_data.class_indices.items()) 

dict_items([('cats', 0), ('dogs', 1)])
dict_items([('cats', 0), ('dogs', 1)])


## 모델 컴파일 및 학습

In [14]:
model.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(2e-5), metrics=['accuracy'])
# Fine-Tuning은 학습율을 낮게(2e-5) 설정해서 pre-trained weights를 조금씩 업데이트 해주는 것이 핵심  

In [ ]:
from datetime import datetime
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

save_file_name = './cats_and_dogs_filtered_Xception_model.h5'

checkpoint = ModelCheckpoint(save_file_name, monitor='val_loss', verbose=1, save_best_only=True, mode='auto')

earlystopping = EarlyStopping(monitor='val_loss', patience=5)


hist = model.fit(train_data, epochs=30, validation_data=test_data, callbacks=[checkpoint, earlystopping])

Epoch 1/30
63/63 [==============================] - ETA: 0s - loss: 0.5529 - accuracy: 0.6975
Epoch 1: val_loss improved from inf to 0.29042, saving model to .\cats_and_dogs_filtered_Xception_model.h5
63/63 [==============================] - 248s 4s/step - loss: 0.5529 - accuracy: 0.6975 - val_loss: 0.2904 - val_accuracy: 0.9290
Epoch 2/30
63/63 [==============================] - ETA: 0s - loss: 0.2665 - accuracy: 0.9425
Epoch 2: val_loss improved from 0.29042 to 0.08353, saving model to .\cats_and_dogs_filtered_Xception_model.h5
63/63 [==============================] - 239s 4s/step - loss: 0.2665 - accuracy: 0.9425 - val_loss: 0.0835 - val_accuracy: 0.9780
Epoch 3/30
63/63 [==============================] - ETA: 0s - loss: 0.1404 - accuracy: 0.9690
Epoch 3: val_loss improved from 0.08353 to 0.05767, saving model to .\cats_and_dogs_filtered_Xception_model.h5
63/63 [==============================] - 241s 4s/step - loss: 0.1404 - accuracy: 0.9690 - val_loss: 0.0577 - val_accuracy: 0.9800

63/63 [==============================] - ETA: 0s - loss: 0.0083 - accuracy: 1.0000
Epoch 27: val_loss did not improve from 0.03718
63/63 [==============================] - 619s 10s/step - loss: 0.0083 - accuracy: 1.0000 - val_loss: 0.0430 - val_accuracy: 0.9840
Epoch 28/30
48/63 [=====================>........] - ETA: 2:10 - loss: 0.0099 - accuracy: 0.9987

## 손실 및 정확도

In [ ]:
import matplotlib.pyplot as plt

plt.title('loss trend')
plt.grid()
plt.xlabel('epochs')
plt.ylabel('loss')
plt.plot(hist.history['loss'], label='train')
plt.plot(hist.history['val_loss'], label='validation')
plt.legend(loc='best')
plt.show()

In [ ]:
plt.title('accuracy trend')
plt.grid()
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.plot(hist.history['accuracy'], label='train')
plt.plot(hist.history['val_accuracy'], label='validation')
plt.legend(loc='best')
plt.show()

## 테스트 데이터 로드 및 예측

In [ ]:
import cv2
import glob

test_img_list = []

test_img_name_list = glob.glob('') # 테스트 이미지 파일이름 리스트

for i in range(len(test_img_name_list)):
    
    # 테스트 이미지 파일 로드 및 전처리
    src_img = cv2.imread(test_img_name_list[i], cv2.IMREAD_COLOR) # 이미지 불러오기
    src_img = cv2.resize(src_img, dsize=(img_width, img_height))  # 사이즈 조절
    
    dst_img = c2.cvtColor(src_img, cv2.COLOR_BGR2RGB) # 컬러 조절?
    dst_img = dst_img / 255.0 # 정규화
    
    test_img_list.append(dst_img)

In [ ]:
plt.figure(figsize=(8, 6))

for i in range(len(test_img_list)):
    
    plt.subplot(2, 3, i+1)
    plt.axis('off')
    plt.imshow(test_img_list[i])
    
plt.show()

In [ ]:
for i in range(len(test_img_list)):
    
    print(test_img_list[i].shape)

In [ ]:
import numpy as np

pred = model.predict(np.array(test_img_list)) # batch차원을 추가하여 predict실행(batch_size, width, height, channel)

class_name = ['cat','dog'] # ImageDataGenerator 에서 cat = 0, dog = 1

plt.figure(figsize=(8, 6))

for i in range(len(pred)):
    
    plt.subplot(2, 3, i+1)
    prediction = str(class_name[np.argmax(pred[i])])
    probility = '{0:0.2f}'.format(100*max(pred[i]))
    title_str = prediction + ' , ' + probility + '%'
    plt.axis('off')
    plt.title(tite_str)
    plt.imshow(test_img_list[i])
    
plt.show()